In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Libraries

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CPU times: user 1.46 s, sys: 634 ms, total: 2.09 s
Wall time: 2.96 s


## Tamil Dataset 

In [ ]:
%%time
# ekhane traind data te 2ta unlabeld data chilo 1186 & 1924 no index e. jegula respectively Hope-Speech and Misandry diye replace kora hoise.
train_data =  pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/ta-misogyny-train-labeled.csv')
valid_data = pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/ta-misogyny-dev-labeled.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab1/ta-misogyny-test(0).csv')
#print(test_data.head())
#print(test_data.shape)

CPU times: user 42.7 ms, sys: 15.1 ms, total: 57.8 ms
Wall time: 1.94 s


In [ ]:
train_data['Class'].value_counts()

None-of-the-above    1296
Misandry              447
Counter-speech        149
Misogyny              125
Xenophobia             95
Hope-Speech            87
Homophobia             35
Transphobic             6
Name: Class, dtype: int64

In [ ]:
valid_data['Class'].value_counts()

None-of-the-above    346
Misandry             104
Counter-speech        36
Xenophobia            29
Misogyny              24
Hope-Speech           11
Homophobia             8
Transphobic            2
Name: Class, dtype: int64

In [ ]:
valid_data.head(2)

,Comment,Class
0,\tமனைவியை விரும்பினால் சுவாரசியம் வரும்,None-of-the-above
1,\tமதுவாந்தி ஒரு கருங்காலி பெண்ணினத்திற்கே ஒரு ...,Misogyny


In [ ]:
# train_data = train_data.drop(columns=2)
# train_data = train_data.rename(columns={0:'text',1:'label'})
# valid_data = valid_data.drop(columns=2)
# valid_data = valid_data.rename(columns={0:'text',1:'label'})
# test_data = test_data.rename(columns={0:'text'})

In [ ]:
train_data['enc_Class'] = train_data['Class'].replace({'None-of-the-above':0, 'Misogyny':1,'Misandry':2,'Homophobia':3,'Transphobic':4,'Xenophobia':5,'Counter-speech':6,'Hope-Speech':7})
valid_data['enc_Class'] = valid_data['Class'].replace({'None-of-the-above':0, 'Misogyny':1,'Misandry':2,'Homophobia':3,'Transphobic':4,'Xenophobia':5,'Counter-speech':6,'Hope-Speech':7})

In [ ]:
valid_data.head(1)

,Comment,Class,enc_Class
0,\tமனைவியை விரும்பினால் சுவாரசியம் வரும்,None-of-the-above,0


In [ ]:
train_data['enc_Class'].value_counts()

0    1296
2     447
6     149
1     125
5      95
7      87
3      35
4       6
Name: enc_Class, dtype: int64

In [ ]:
valid_data['enc_Class'].value_counts()

0    346
2    104
6     36
5     29
1     24
7     11
3      8
4      2
Name: enc_Class, dtype: int64

In [ ]:
print("Size of the training data ==> ", train_data.shape)
print("Size of the validation data ==>", valid_data.shape)
print("Size of the Test data ==>", test_data.shape)

Size of the training data ==>  (2240, 3)
Size of the validation data ==> (560, 3)
Size of the Test data ==> (699, 2)


## One Hot Encoding

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder=OneHotEncoder(sparse=False)

encoded_Classes = pd.DataFrame (encoder.fit_transform(train_data[['Class']]))

encoded_Classes .columns = encoder.get_feature_names(['Class'])

train_data= pd.concat([train_data, encoded_Classes ], axis=1)

In [ ]:
train_data.head(1)

,Comment,Class,enc_Class,Class_Counter-speech,Class_Homophobia,Class_Hope-Speech,Class_Misandry,Class_Misogyny,Class_None-of-the-above,Class_Transphobic,Class_Xenophobia
0,\tஉங்கள் பேச்சை எதிர்பார்த்தேன். நல்ல விளக்கம்...,Hope-Speech,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_data = train_data.rename(columns={'Class_None-of-the-above':'None-of-the-above', 'Class_Misogyny':'Misogyny','Class_Misandry':'Misandry','Class_Homophobia':'Homophobia','Class_Transphobic':'Transphobic','Class_Xenophobia':'Xenophobia','Class_Counter-speech':'Counter-speech','Class_Hope-Speech':'Hope-Speech'})#'Neutral':0, 'Positive':1,'Highly Negative':2,'Negative':3,'Highly Positive':4
train_data.head(1)

,Comment,Class,enc_Class,Counter-speech,Homophobia,Hope-Speech,Misandry,Misogyny,None-of-the-above,Transphobic,Xenophobia
0,\tஉங்கள் பேச்சை எதிர்பார்த்தேன். நல்ல விளக்கம்...,Hope-Speech,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Cleaning

In [ ]:
'''
Text Cleaning
'''
def text_cleaning(row):
  # to remove HTML tags
  row=str(row)
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  #pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  #text = re.sub(pat, '', new_text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower() 
  # Stopword Removing
  tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  # check in lowercase 
  #t = [token for token in tokens if token.lower() not in stopword_list]
  #text = ' '.join(t) 
  # Stemming
  #t = [stemming.stem(token) for token in tokens ]
  #text = ' '.join(t) 

  return text



In [ ]:
# def get_tamil_stop_words():
#     swdf1 = pd.read_csv("/content/drive/MyDrive/automated_tamil_text_classification-master/data/TamilNLP_TamilStopWords.txt",  header=None) 
#     swdf2 = pd.read_csv("/content/drive/MyDrive/automated_tamil_text_classification-master/data/custom_tamil_stop_words.txt",  header=None) 
#      sw1 = swdf1[0].tolist()
#     sw2 = swdf2[0].tolist()
#     tamil_stop_words = list(set(sw1 + sw2))
#     return tamil_stop_words

In [ ]:
#removing URLs and Punctuations
#train_data['cleaned'] = train_data['tweet'].apply(remove_url)
# #Removing punctuations
# tamil_stop_words = get_tamil_stop_words()
# valid_data['Texts'] = valid_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (tamil_stop_words)]))
# train_data['Texts'] = train_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (tamil_stop_words)]))
# test_data['Texts'] = test_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (tamil_stop_words)]))


train_data['cleaned'] = train_data['Comment'].apply(text_cleaning)
valid_data['cleaned'] = valid_data['Comment'].apply(text_cleaning)
test_data['cleaned'] = test_data['Comment'].apply(text_cleaning)


## Data samples after cleaning
print("Data samples after cleaning:\n")
for i in range(1):
  print("Original Data:===\n",train_data.Comment[i],"\nCleaned Data:===\n",train_data.cleaned[i],)

Data samples after cleaning:

Original Data:===
 	உங்கள் பேச்சை எதிர்பார்த்தேன். நல்ல விளக்கம்  அருமை.  நியாயமான முறையில் பதிவு செய்துள்ளீர்கள்.  நீண்ட காலம் வாழ்வதற்கு வாழ்த்துக்கள். 
Cleaned Data:===
 உங்கள் பேச்சை எதிர்பார்த்தேன் நல்ல விளக்கம் அருமை நியாயமான முறையில் பதிவு செய்துள்ளீர்கள் நீண்ட காலம் வாழ்வதற்கு வாழ்த்துக்கள்


In [ ]:
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(2240, 12)
(560, 4)
(699, 3)


In [ ]:
valid_data['cleaned'].shape

(560,)

## Data Statistics

In [ ]:
def char_length(row):
    return len(row.split())

In [ ]:
Classes = ['None-of-the-above', 'Misogyny','Misandry','Homophobia','Transphobic','Xenophobia','Counter-speech','Hope-Speech']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in encoded_Classes:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ") 
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    

In [ ]:
train_data.head(1)

,Comment,Class,enc_Class,Counter-speech,Homophobia,Hope-Speech,Misandry,Misogyny,None-of-the-above,Transphobic,Xenophobia,cleaned
0,\tஉங்கள் பேச்சை எதிர்பார்த்தேன். நல்ல விளக்கம்...,Hope-Speech,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,உங்கள் பேச்சை எதிர்பார்த்தேன் நல்ல விளக்கம் அர...


In [ ]:
# dataset_statistics(train_data)

##Machine Learning

In [ ]:
def print_metrices(pred,true):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,))
    print("Accuracy : ",accuracy_score(pred,true))
    print("Precison : ",precision_score(pred,true, average = 'weighted'))
    print("Recall : ",recall_score(pred,true,  average = 'weighted'))
    print("F1 : ",f1_score(pred,true,  average = 'weighted'))

In [ ]:
## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
#use_idf=True,tokenizer=lambda x: x.split()
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split())
vectorised_posts = vectorizer.fit_transform(train_data['cleaned'])
vectorized_train_posts = vectorizer.transform(train_data['cleaned'])
vectorized_valid_posts = vectorizer.transform(valid_data['cleaned'])
vectorized_test_posts = vectorizer.transform(test_data['cleaned'])

print("Feature Size: ", vectorized_train_posts.shape[1])
#vectorizer.vocabulary_

Feature Size:  11275


In [ ]:
vectorized_valid_posts.shape

(560, 11275)

In [ ]:
valid_data['Class'].value_counts()

None-of-the-above    346
Misandry             104
Counter-speech        36
Xenophobia            29
Misogyny              24
Hope-Speech           11
Homophobia             8
Transphobic            2
Name: Class, dtype: int64

In [ ]:
Classes =  ['None-of-the-above', 'Misogyny','Misandry','Homophobia','Transphobic','Xenophobia','Counter-speech','Hope-Speech']
def actual_Class(row):
    return Classes[row]

### Logistic Regression

In [ ]:
# After Tunning
# Optimum C = 100 
lr = LogisticRegression(class_weight='balanced',C =2)
lr.fit(vectorized_train_posts,train_data['enc_Class'])
y_pred = lr.predict(vectorized_valid_posts)
print_metrices(y_pred,valid_data['enc_Class'])

[[237   8  34   4   0   6  38  19]
 [  7   7   4   4   0   0   2   0]
 [ 24   4  68   0   0   4   3   1]
 [  2   0   4   2   0   0   0   0]
 [  0   1   1   0   0   0   0   0]
 [ 10   0   9   0   0   9   1   0]
 [ 15   3   1   0   0   0  17   0]
 [  6   0   1   0   0   0   1   3]]
              precision    recall  f1-score   support

           0       0.79      0.68      0.73       346
           1       0.30      0.29      0.30        24
           2       0.56      0.65      0.60       104
           3       0.20      0.25      0.22         8
           4       0.00      0.00      0.00         2
           5       0.47      0.31      0.38        29
           6       0.27      0.47      0.35        36
           7       0.13      0.27      0.18        11

    accuracy                           0.61       560
   macro avg       0.34      0.37      0.34       560
weighted avg       0.65      0.61      0.63       560

Accuracy :  0.6125
Precison :  0.6010730740272175
Recall :  0.6125
F

In [ ]:
test_pred = lr.predict(vectorized_test_posts).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
run = pd.DataFrame(test_data['id'])
# run['text'] = test_data['Text']
run['Class'] = test_pred
run['Class'] = run['Class'].apply(actual_Class)
run 

,id,Class
0,1,Xenophobia
1,2,Counter-speech
2,3,Misandry
3,4,Counter-speech
4,5,None-of-the-above
...,...,...
694,695,None-of-the-above
695,696,None-of-the-above
696,697,Counter-speech
697,698,Misandry


In [ ]:
run['Class'].value_counts()

None-of-the-above    348
Misandry             141
Counter-speech        84
Misogyny              52
Hope-Speech           44
Xenophobia            16
Homophobia            13
Transphobic            1
Name: Class, dtype: int64

In [ ]:
folder_path="/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/"

In [ ]:
run.to_csv(folder_path +'tamil_lr.tsv',sep = '\t',index = False)

## Decision Tree


In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced')
dt.fit(vectorized_train_posts,train_data['enc_Class'])
y_pred = dt.predict(vectorized_valid_posts)
print_metrices(y_pred,valid_data['enc_Class'])

[[251  11  35   4   1   3  15  26]
 [ 12   4   2   4   0   0   2   0]
 [ 36   4  54   1   0   4   3   2]
 [  2   0   3   3   0   0   0   0]
 [  0   0   1   1   0   0   0   0]
 [  8   1  10   0   0   7   1   2]
 [ 17   4   2   0   0   1   9   3]
 [  8   1   0   0   0   0   0   2]]
              precision    recall  f1-score   support

           0       0.75      0.73      0.74       346
           1       0.16      0.17      0.16        24
           2       0.50      0.52      0.51       104
           3       0.23      0.38      0.29         8
           4       0.00      0.00      0.00         2
           5       0.47      0.24      0.32        29
           6       0.30      0.25      0.27        36
           7       0.06      0.18      0.09        11

    accuracy                           0.59       560
   macro avg       0.31      0.31      0.30       560
weighted avg       0.61      0.59      0.60       560

Accuracy :  0.5892857142857143
Precison :  0.5792472905018375
Recall

In [ ]:
test_pred = dt.predict(vectorized_test_posts).tolist()
test_data['id'] = test_data.index + 1
test_pred

[2,
 0,
 2,
 2,
 0,
 6,
 1,
 3,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 7,
 0,
 0,
 7,
 0,
 6,
 0,
 2,
 2,
 2,
 0,
 6,
 0,
 3,
 0,
 2,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 6,
 6,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 7,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 0,
 6,
 2,
 0,
 0,
 6,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 6,
 0,
 0,
 6,
 2,
 0,
 0,
 7,
 0,
 0,
 5,
 0,
 6,
 0,
 7,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 6,
 6,
 1,
 2,
 0,
 6,
 0,
 0,
 0,
 7,
 2,
 2,
 2,
 0,
 6,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 6,
 0,
 0,
 0,
 0,
 2,
 7,
 0,
 0,
 0,
 5,
 6,
 0,
 6,
 2,
 6,
 0,
 0,
 1,
 6,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 7,
 0,
 2,
 0,
 0,
 7,
 0,
 1,
 0,
 3,
 0,
 2,
 0,
 7,
 2,
 2,
 2,
 6,
 2,
 0,
 0,
 0,
 0,
 7,
 2,
 2,
 7,
 2,
 2,
 1,
 6,
 7,
 2,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 6,
 2,
 6,
 0,
 2,
 0,
 0,
 0,
 5,
 0,
 2,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 2,
 1,
 0,
 2,
 6,
 7,
 6,
 0,
 0,
 5,
 5,
 0,
 0,


In [ ]:
run = pd.DataFrame(test_data['id'])
# run['text'] = test_data['Text']
run['Class'] = test_pred
run['Class'] = run['Class'].apply(actual_Class)
run 

,id,Class
0,1,Misandry
1,2,None-of-the-above
2,3,Misandry
3,4,Misandry
4,5,None-of-the-above
...,...,...
694,695,None-of-the-above
695,696,Misandry
696,697,Counter-speech
697,698,Xenophobia


In [ ]:
run.to_csv(folder_path +'tamil_DT.tsv',sep = '\t',index = False)

### Support Vector Machine

In [ ]:
## SVM
from sklearn.svm import SVC
svc =  SVC(C= 2,random_state=4,class_weight='balanced')
svc.fit(vectorized_train_posts,train_data['enc_Class'])
y_pred = svc.predict(vectorized_valid_posts)
print_metrices(y_pred,valid_data['enc_Class'])

[[318   2  19   3   0   0   2   2]
 [ 17   3   3   0   0   0   1   0]
 [ 47   0  57   0   0   0   0   0]
 [  6   0   1   1   0   0   0   0]
 [  0   1   1   0   0   0   0   0]
 [ 18   0   9   0   0   2   0   0]
 [ 34   0   1   0   0   0   1   0]
 [  9   0   0   0   0   0   0   2]]
              precision    recall  f1-score   support

           0       0.71      0.92      0.80       346
           1       0.50      0.12      0.20        24
           2       0.63      0.55      0.58       104
           3       0.25      0.12      0.17         8
           4       0.00      0.00      0.00         2
           5       1.00      0.07      0.13        29
           6       0.25      0.03      0.05        36
           7       0.50      0.18      0.27        11

    accuracy                           0.69       560
   macro avg       0.48      0.25      0.27       560
weighted avg       0.66      0.69      0.63       560

Accuracy :  0.6857142857142857
Precison :  0.8299393834951067
Recall

In [ ]:
test_pred = svc.predict(vectorized_test_posts).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
run = pd.DataFrame(test_data['id'])
# run['text'] = test_data['Text']
run['Class'] = test_pred
run['Class'] = run['Class'].apply(actual_Class)
run 

,id,Class
0,1,None-of-the-above
1,2,None-of-the-above
2,3,Misandry
3,4,None-of-the-above
4,5,None-of-the-above
...,...,...
694,695,None-of-the-above
695,696,None-of-the-above
696,697,None-of-the-above
697,698,Misandry


In [ ]:
run['Class'].value_counts()

None-of-the-above    572
Misandry             101
Misogyny               9
Counter-speech         7
Homophobia             5
Hope-Speech            3
Xenophobia             2
Name: Class, dtype: int64

In [ ]:
run.to_csv(folder_path +'tamil_svm.tsv',sep = '\t',index = False)

##Deep Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
print("Tensorflow Version:--",tf.version.VERSION)
print("Keras Version:---",keras.__version__)

Tensorflow Version:-- 2.8.0
Keras Version:--- 2.8.0


###Tokenization

In [ ]:
tokenizer = Tokenizer(num_words = 10000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='<oov>', document_count=0)
tokenizer.fit_on_texts(train_data['Comment'])  
vocab_size = len(tokenizer.word_index)+1

In [ ]:
print(vocab_size)

11184


In [ ]:
max_len = 260
# Training Sequences
train_sequences = tokenizer.texts_to_sequences(train_data['Comment'])
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)

# Validation Sequences
validation_sequences = tokenizer.texts_to_sequences(valid_data['Comment'])
validation_pad_sequences =  keras.preprocessing.sequence.pad_sequences(validation_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,validation_pad_sequences.shape)

11183
Number of Training Sequences : (2240, 260)
Number of Validation Sequences : (560, 260)


In [ ]:
# Test Sequences
test_sequences = tokenizer.texts_to_sequences((test_data['Comment']))
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

Number of Test Sequences : (699, 260)


In [ ]:
#print(len(str(test_data['Comment'])))
#test_text_list = list(test_data['Comment'])
#print(len(test_text_list))
#testsequ = tokenizer.texts_to_sequences(test_text_list)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes =np.unique(train_data['enc_Class'])
                                               ,y=train_data['enc_Class'])

weight = {i : class_weights[i] for i in range(8)}

In [ ]:
weight

{0: 0.21604938271604937,
 1: 2.24,
 2: 0.6263982102908278,
 3: 8.0,
 4: 46.666666666666664,
 5: 2.9473684210526314,
 6: 1.8791946308724832,
 7: 3.218390804597701}

In [ ]:
folder_path="/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/DL_Models/"

In [ ]:
num_classes = 8

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy') is not None and logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"tamil_BiLSTMModel.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor=f1_m, verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
tf.keras.layers.Conv1D(128, 8, activation='relu'),
tf.keras.layers.MaxPooling1D(8),
tf.keras.layers.Bidirectional(LSTM(units = 128,return_sequences=True,dropout = 0.2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(8 , activation='softmax')])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 260, 100)          1000000   
                                                                 
 conv1d_4 (Conv1D)           (None, 253, 128)          102528    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 31, 128)          0         
 1D)                                                             
                                                                 
 bidirectional_5 (Bidirectio  (None, 31, 256)          263168    
 nal)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 7936)              0         
                                                                 
 dense_5 (Dense)             (None, 8)                

In [ ]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=[f1_m])
   
history = model.fit(train_pad_sequences,
    train_data['enc_Class'],
    epochs=15,
    batch_size = 32,
    validation_split = 0.2,
    verbose =1,
    callbacks = callback_list)
   # class_weight = weight)

Epoch 1/15
56/56 [==============================] - 14s 59ms/step - loss: 1.5543 - f1_m: 2.9947 - val_loss: 1.3308 - val_f1_m: 0.8543
Epoch 2/15
56/56 [==============================] - 2s 40ms/step - loss: 1.3169 - f1_m: 0.8583 - val_loss: 1.3290 - val_f1_m: 0.9954
Epoch 3/15
56/56 [==============================] - 2s 39ms/step - loss: 1.3096 - f1_m: 0.8927 - val_loss: 1.3110 - val_f1_m: 0.9245
Epoch 4/15
56/56 [==============================] - 2s 39ms/step - loss: 1.2888 - f1_m: 0.9590 - val_loss: 1.3002 - val_f1_m: 1.0594
Epoch 5/15
56/56 [==============================] - 2s 38ms/step - loss: 1.2613 - f1_m: 1.0605 - val_loss: 1.2905 - val_f1_m: 1.2147
Epoch 6/15
56/56 [==============================] - 2s 38ms/step - loss: 1.2291 - f1_m: 1.1643 - val_loss: 1.2893 - val_f1_m: 1.0923
Epoch 7/15
56/56 [==============================] - 2s 38ms/step - loss: 1.1800 - f1_m: 1.2689 - val_loss: 1.2969 - val_f1_m: 1.2915
Epoch 8/15
56/56 [==============================] - 2s 38ms/step - l

In [ ]:

# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)
print_metrices(y_pred,valid_data['enc_Class'])


[[245   8  87   0   0   1   0   5]
 [ 10   0  11   0   0   3   0   0]
 [ 39   2  54   0   0   4   0   5]
 [  4   1   3   0   0   0   0   0]
 [  1   0   1   0   0   0   0   0]
 [  5   4  17   0   0   2   0   1]
 [  9   4  17   0   0   2   1   3]
 [  6   1   3   0   0   0   0   1]]
              precision    recall  f1-score   support

           0       0.77      0.71      0.74       346
           1       0.00      0.00      0.00        24
           2       0.28      0.52      0.36       104
           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00         2
           5       0.17      0.07      0.10        29
           6       1.00      0.03      0.05        36
           7       0.07      0.09      0.08        11

    accuracy                           0.54       560
   macro avg       0.29      0.18      0.17       560
weighted avg       0.60      0.54      0.53       560

Accuracy :  0.5410714285714285
Precison :  0.5862715030357546
Recall

### Test Set Prediction

In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()

In [ ]:
test_data['id'] = test_data.index + 1
run = pd.DataFrame(test_data['id'])
len(test_pad_sequences)
run['Comment'] = test_data['Comment']
run['Class'] = test_pred
run['Class'] = run['Class'].apply(actual_Class)
run 

,id,Comment,Class
0,1,அந்த தமிழன் ஒரு சாதிவெறி,Misogyny
1,2,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...,Misogyny
2,3,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,4,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...,Misogyny
4,5,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...,Hope-Speech
...,...,...,...
694,695,சினிவாசன் அய்யாக்கு நன்றி...,None-of-the-above
695,696,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை,None-of-the-above
696,697,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...,Counter-speech
697,698,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...,Xenophobia


In [ ]:
run.to_csv(folder_path +'afterSW_tamil_cnn+bilstm22.tsv',sep = '\t',index = False)

In [ ]:
run.to_csv(folder_path +'tamil_cnn+bilstm22.tsv',sep = '\t',index = False)

In [ ]:
run.to_csv(folder_path +'mal_cnn+bilstm.tsv',sep = '\t',index = False)

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/DL_Models/afterSW_tamil_cnn+bilstm22.tsv')
df2=pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/DL_Models/tamil_cnn+bilstm22.tsv')
df3=pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/DL_Models/mal_cnn+bilstm.tsv')

In [ ]:
df2.equals(df3)

True

##Pretrained Word Embedding (Fasttext)

In [ ]:
from urllib.request import urlopen
import gzip

# get the vectors
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ta.300.vec.gz'))

In [ ]:
tokenizer = Tokenizer(num_words = 10000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='<oov>', document_count=0)
tokenizer.fit_on_texts(train_data['Comment'])  
word_index = tokenizer.word_index
vocab_size = len(word_index )+1

In [ ]:
vocab_and_vectors = {}
# put words as dict indexes and vectors as words values
for line in file:
  values = line.split()
  word = values [0].decode('utf-8')
  vector = np.asarray(values[1:], dtype='float32')
  vocab_and_vectors[word] = vector

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in word_index.items():
  embedding_vector = vocab_and_vectors.get(word)
  # words that cannot be found will be set to 0
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
max_len = 260
# Training Sequences
train_sequences = tokenizer.texts_to_sequences(train_data['Comment'])
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)

# Validation Sequences
validation_sequences = tokenizer.texts_to_sequences(valid_data['Comment'])
validation_pad_sequences =  keras.preprocessing.sequence.pad_sequences(validation_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,validation_pad_sequences.shape)

11183
Number of Training Sequences : (2240, 260)
Number of Validation Sequences : (560, 260)


In [ ]:
# Test Sequences
test_sequences = tokenizer.texts_to_sequences(test_data['Comment'])
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

Number of Test Sequences : (699, 260)


In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes =np.unique(train_data['enc_Class'])
                                               ,y=train_data['enc_Class'])

weight = {i : class_weights[i] for i in range(8)}

#### Model Defination

In [ ]:
num_classes = 8

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy') is not None and logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"prt_ta_BiLSTMModel.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor=f1_m, verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(vocab_size, 300,weights=[embedding_matrix],trainable=False, input_length = max_len),
tf.keras.layers.Conv1D(128, 8, activation='relu'),
tf.keras.layers.Conv1D(128, 8, activation='relu'),
tf.keras.layers.MaxPooling1D(8),
tf.keras.layers.Bidirectional(LSTM(units = 128,return_sequences=True,dropout = 0.2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(8 , activation='softmax')])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 260, 300)          3355200   
                                                                 
 conv1d (Conv1D)             (None, 253, 128)          307328    
                                                                 
 conv1d_1 (Conv1D)           (None, 246, 128)          131200    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 128)          0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 30, 256)          263168    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 7680)              0

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=[f1_m])
   
history = model.fit(train_pad_sequences,
    train_data['enc_Class'],
    epochs=25,
    batch_size = 32,
    validation_split = 0.2,
    verbose =1,
    callbacks = callback_list)
    # class_weight = weight)

Epoch 1/25
56/56 [==============================] - 21s 297ms/step - loss: 1.4312 - f1_m: 2.0460 - val_loss: 1.3057 - val_f1_m: 1.0391
Epoch 2/25
56/56 [==============================] - 15s 272ms/step - loss: 1.2654 - f1_m: 1.2377 - val_loss: 1.1925 - val_f1_m: 1.0848
Epoch 3/25
56/56 [==============================] - 15s 277ms/step - loss: 1.1081 - f1_m: 1.1644 - val_loss: 1.1399 - val_f1_m: 1.3465
Epoch 4/25
56/56 [==============================] - 15s 271ms/step - loss: 0.9891 - f1_m: 1.1481 - val_loss: 1.1346 - val_f1_m: 0.9872
Epoch 5/25
56/56 [==============================] - 15s 276ms/step - loss: 0.8374 - f1_m: 1.0574 - val_loss: 1.2713 - val_f1_m: 1.0502
Epoch 6/25
56/56 [==============================] - 15s 273ms/step - loss: 0.6982 - f1_m: 0.9578 - val_loss: 1.2896 - val_f1_m: 1.0243
Epoch 7/25
56/56 [==============================] - 15s 277ms/step - loss: 0.5047 - f1_m: 0.8926 - val_loss: 1.5627 - val_f1_m: 0.8020
Epoch 8/25
56/56 [==============================] - 15s

In [ ]:

y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)
print_metrices(y_pred,valid_data['enc_Class'])


[[301   1  22   2   0   2  13   5]
 [ 13   4   6   0   0   0   1   0]
 [ 53   3  46   0   0   2   0   0]
 [  4   1   2   1   0   0   0   0]
 [  1   0   1   0   0   0   0   0]
 [ 18   0   5   0   0   4   2   0]
 [ 20   3   1   0   0   1  11   0]
 [  9   0   0   0   0   0   1   1]]
              precision    recall  f1-score   support

           0       0.72      0.87      0.79       346
           1       0.33      0.17      0.22        24
           2       0.55      0.44      0.49       104
           3       0.33      0.12      0.18         8
           4       0.00      0.00      0.00         2
           5       0.44      0.14      0.21        29
           6       0.39      0.31      0.34        36
           7       0.17      0.09      0.12        11

    accuracy                           0.66       560
   macro avg       0.37      0.27      0.29       560
weighted avg       0.62      0.66      0.63       560

Accuracy :  0.6571428571428571
Precison :  0.7391691218209182
Recall

In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
test_pred

[0,
 6,
 2,
 6,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 7,
 0,
 0,
 6,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 6,
 0,
 6,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 6,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 6,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 6,
 2,
 2,
 6,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 2,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 6,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 6,
 0,
 0,
 0,
 2,
 6,
 0,
 0,


In [ ]:
test_data['id'] = test_data.index + 1
run = pd.DataFrame(test_data['id'])
run['Comment'] = test_data['Comment']
run['Class'] = test_pred
run['Class'] = run['Class'].apply(actual_Class)
run 


,id,Comment,Class
0,1,அந்த தமிழன் ஒரு சாதிவெறி,None-of-the-above
1,2,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...,Counter-speech
2,3,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,4,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...,Counter-speech
4,5,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...,None-of-the-above
...,...,...,...
694,695,சினிவாசன் அய்யாக்கு நன்றி...,None-of-the-above
695,696,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை,None-of-the-above
696,697,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...,Counter-speech
697,698,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...,None-of-the-above


In [ ]:
run.to_csv(folder_path +'afterSW_tamil_cnn+bilstm+fasttext.tsv',sep = '\t',index = False)

In [ ]:
folder_path

'/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/DL_Models/'

In [ ]:
run.to_csv(folder_path +'mal_cnn+bilstm+fasttext.tsv',sep = '\t',index = False)

In [ ]:
a=pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/DL_Models/afterSW_tamil_cnn+bilstm+fasttext.tsv')
b=pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/DL_Models/mal_cnn+bilstm+fasttext.tsv')
a.equals(b)

True

### Ensemble Appraoch (Majority Voting)

In [ ]:

LogReg_clf = LogisticRegression(class_weight='balanced')
DTree_clf = DecisionTreeClassifier(class_weight='balanced')
rt_clf =   RandomForestClassifier(class_weight='balanced')
SVC_clf = SVC(C = 6,class_weight='balanced')


voting_clf = VotingClassifier(estimators=[('SVC', SVC_clf), ('DTree', DTree_clf), ('LogReg', LogReg_clf),('RF', rt_clf)], voting='hard')
voting_clf.fit(vectorized_train_posts, train_data['enc_Class'])
preds = voting_clf.predict(vectorized_valid_posts)
print_metrices(valid_data['enc_Class'], preds)


[[328  18  49   3   0  19  34   9]
 [  2   3   1   0   1   1   1   0]
 [ 10   2  54   3   1   7   0   0]
 [  3   1   0   2   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   2   0   0]
 [  1   0   0   0   0   0   1   0]
 [  1   0   0   0   0   0   0   2]]
              precision    recall  f1-score   support

           0       0.95      0.71      0.81       460
           1       0.12      0.33      0.18         9
           2       0.52      0.70      0.60        77
           3       0.25      0.33      0.29         6
           4       0.00      0.00      0.00         0
           5       0.07      0.67      0.12         3
           6       0.03      0.50      0.05         2
           7       0.18      0.67      0.29         3

    accuracy                           0.70       560
   macro avg       0.27      0.49      0.29       560
weighted avg       0.86      0.70      0.76       560

Accuracy :  0.7
Precison :  0.6696097174047484
Recall :  0.7
F1 :  0

In [ ]:
test_pred = voting_clf.predict(vectorized_test_posts).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
run = pd.DataFrame(test_data['id'])
run['Comment'] = test_data['Comment']
run['Class'] = test_pred
run['Class'] = run['Class'].apply(actual_Class)
run 

,id,Comment,Class
0,1,அந்த தமிழன் ஒரு சாதிவெறி,None-of-the-above
1,2,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...,None-of-the-above
2,3,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,4,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...,None-of-the-above
4,5,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...,None-of-the-above
...,...,...,...
694,695,சினிவாசன் அய்யாக்கு நன்றி...,None-of-the-above
695,696,ஊருக்கே தெரியும் அவர் பிராமணன் என்று.அவர் ஒரு தலை,None-of-the-above
696,697,ஐயா அவர்கள் சரியான விளக்கம் பார்பனர்கள் வேறு ப...,None-of-the-above
697,698,Nandha Kumar அட நாயே; ; வந்தேறி பிராமண நாய...,Misandry


In [ ]:
import pandas as pd
a = pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/tamil_DT.tsv')
b = pd.read_csv('/content/drive/MyDrive/Shared_Task_Dataset/Codalab2/output-onlinetsvtools.tsv')

In [ ]:
a.equals(b)

True

In [ ]:
run['Class'].value_counts()

None-of-the-above    589
Misandry              84
Misogyny              11
Homophobia             6
Hope-Speech            4
Xenophobia             3
Counter-speech         2
Name: Class, dtype: int64

In [ ]:
run.to_csv(folder_path +'mal_ensemble.tsv',sep = '\t',index = False)

# New section